# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("../WeatherPy/output_data/cities.csv")
data.head()

,Unnamed: 0,City,Lng,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saldanha,17.9442,-33.0117,68.00,77,100,13.80,ZA,1611430737
1,1,Dunedin,170.5036,-45.8742,59.00,67,98,5.01,NZ,1611430737
2,2,Uuemõisa,23.5894,58.9394,35.01,90,100,5.01,EE,1611430737
3,3,Alugan,125.4808,12.2188,78.53,84,56,4.65,PH,1611430737
4,4,Busselton,115.3333,-33.6500,61.00,88,85,1.97,AU,1611430737


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)
marker_locations = data[["Lat", "Lng"]]
humidity=data["Humidity"]
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=5)
fig=gmaps.figure()
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [4]:
fit_weather=data.loc[(data["Max Temp"]>75) & (data["Max Temp"]<85) & (data["Wind Speed"]<10) & (data["Cloudiness"]==0)]
fit_weather

,Unnamed: 0,City,Lng,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,91,Saint-Philippe,55.7679,-21.3585,80.60,74,0,4.61,RE,1611430748
206,206,Kang,22.7876,-23.6752,79.05,20,0,6.31,BW,1611430763
272,272,Luanda,13.2343,-8.8368,78.80,78,0,6.91,AO,1611430549
308,308,Cap Malheureux,57.6142,-19.9842,79.00,76,0,1.99,MU,1611430776
353,353,Oussouye,-16.5469,12.4850,84.20,65,0,4.61,SN,1611430782
415,415,Kontagora,5.4695,10.3999,77.92,18,0,5.10,NG,1611430791
521,521,Pandan,124.1694,14.0453,77.29,82,0,3.36,PH,1611430810


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Drop null values
hotel_df = fit_weather.dropna()
# Add Hotel Name column
hotel_df["Hotel Name"] = ""
# Set parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# Loop over rows in dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make API request and print url
    hotel_name = requests.get(base_url, params=params)

    # Convert to json
    hotel_name = hotel_name.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,Unnamed: 0,City,Lng,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
91,91,Saint-Philippe,55.7679,-21.3585,80.60,74,0,4.61,RE,1611430748,"Chambres d'hôte ""La Trinité"""
206,206,Kang,22.7876,-23.6752,79.05,20,0,6.31,BW,1611430763,Kang Ultra Stop
272,272,Luanda,13.2343,-8.8368,78.80,78,0,6.91,AO,1611430549,EPIC SANA Luanda
308,308,Cap Malheureux,57.6142,-19.9842,79.00,76,0,1.99,MU,1611430776,Pereybere Hotel & Spa
353,353,Oussouye,-16.5469,12.4850,84.20,65,0,4.61,SN,1611430782,Auberge du Routard
415,415,Kontagora,5.4695,10.3999,77.92,18,0,5.10,NG,1611430791,City Gate Inn
521,521,Pandan,124.1694,14.0453,77.29,82,0,3.36,PH,1611430810,Bonifacio Lodge


In [6]:
hotel_df_coords = hotel_df[["Lat","Lng"]]
hotel_layer = gmaps.marker_layer(hotel_df_coords)

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))